#Install needed libraries

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/"
%cd {path}
!ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1


In [ ]:
!git clone https://github.com/phamdinhkhanh/vnquant
%cd vnquant
!python setup.py install

Cloning into 'vnquant'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 341 (delta 65), reused 89 (delta 53), pack-reused 237
Receiving objects: 100% (341/341), 1.05 MiB | 7.00 MiB/s, done.
Resolving deltas: 100% (164/164), done.
/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/vnquant
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/

In [ ]:
%cd ..
!rm -rf vnquant
!ls

/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1


In [ ]:
!pip freeze | grep vnquant

vnquant==0.1.1


In [ ]:
import vnquant
vnquant.__version__

'0.1.1'

In [ ]:
import vnquant.data as dt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import accuracy_score, classification_report

#We choose 10 codes from biggest companies on stock exchange in Vietnam

In [ ]:
stock_symbols = ['VIC', 'VCB', 'VNM', 'GAS', 'HVN', 'MWG', 'FPT', 'HPG', 'MSN', 'SAB']

#Prepare the dataset

In [ ]:
stock_data = {}

for symbol in stock_symbols:
  df = dt.DataLoader(symbols=symbol,
                     start="2014-01-01",
                     end="2024-01-01",
                     minimal=True,
                     data_source="VND")
  stock_data[symbol] = df.download()

In [ ]:
stock_data[stock_symbols[0]]

Attributes,high,low,open,close,avg,volume
Symbols,VIC,VIC,VIC,VIC,VIC,VIC
date,,,,,,
2014-01-02,70.50,69.50,70.00,70.00,69.92,74950.0
2014-01-03,70.50,69.50,70.00,70.50,70.00,91920.0
2014-01-06,70.50,69.50,70.50,70.50,70.11,111760.0
2014-01-07,70.50,70.00,70.50,70.50,70.47,294120.0
2014-01-08,70.50,70.00,70.00,70.00,70.02,109890.0
...,...,...,...,...,...,...
2023-12-25,43.55,43.00,43.10,43.40,43.34,3364500.0
2023-12-26,43.75,43.35,43.40,43.55,43.55,1806700.0


In [ ]:
def reconstruct_data(list_price):
  week = []
  combine = []
  i = 0
  while (i < len(list_price)):
    if (len(week) != 4):
      week.append(np.round(np.mean(list_price[i:i+7]), 2))
      i = i + 7
    else:
      combine.append(week)
      week = []

  return combine

##VIC

In [ ]:
close = stock_data[stock_symbols[0]]['close'].values
open = stock_data[stock_symbols[0]]['open'].values
high = stock_data[stock_symbols[0]]['high'].values
low = stock_data[stock_symbols[0]]['low'].values
avg = stock_data[stock_symbols[0]]['avg'].values
volume = stock_data[stock_symbols[0]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_vic = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[0]}'), ('close_week2', f'{stock_symbols[0]}'),
                                  ('close_week3', f'{stock_symbols[0]}'), ('close_week4', f'{stock_symbols[0]}')], dtype = np.dtype(np.float64))
df_vic.head()

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)"
0,70.43,72.93,75.93,77.43
1,77.71,74.64,77.14,75.50
2,72.36,66.79,65.93,63.43
3,66.07,67.21,65.21,64.07
4,64.14,69.21,69.86,72.21


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_vic[('close_total4week', f'{stock_symbols[0]}')] = mean_close[:len(df_vic)]
df_vic[('open_total4week', f'{stock_symbols[0]}')] = mean_open[:len(df_vic)]
df_vic[('high_total4week', f'{stock_symbols[0]}')] = mean_high[:len(df_vic)]
df_vic[('low_total4week', f'{stock_symbols[0]}')] = mean_low[:len(df_vic)]
df_vic[('avg_total4week', f'{stock_symbols[0]}')] = mean_avg[:len(df_vic)]
df_vic[('volume_total4week', f'{stock_symbols[0]}')] = mean_volume[:len(df_vic)]

In [ ]:
df_vic

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)"
0,70.43,72.93,75.93,77.43,74.18,74.12,74.82,73.20,74.03,259468.93
1,77.71,74.64,77.14,75.50,76.25,76.62,77.11,75.61,76.32,401583.57
2,72.36,66.79,65.93,63.43,67.12,67.21,68.00,66.20,67.00,669534.64
3,66.07,67.21,65.21,64.07,65.64,65.70,66.29,64.96,65.61,247855.36
4,64.14,69.21,69.86,72.21,68.86,68.93,69.57,68.21,68.87,429015.71
...,...,...,...,...,...,...,...,...,...,...
84,53.06,52.00,50.80,52.00,51.96,52.16,52.52,51.67,52.02,2993792.86
85,52.04,62.04,71.27,64.44,62.45,63.04,63.80,61.65,62.67,13572447.36
86,61.00,55.26,47.25,45.59,52.28,53.05,53.66,51.69,52.57,14991744.82
87,44.86,42.80,41.59,44.69,43.48,43.54,43.98,42.68,43.32,6670760.71


##VCB

In [ ]:
close = stock_data[stock_symbols[1]]['close'].values
open = stock_data[stock_symbols[1]]['open'].values
high = stock_data[stock_symbols[1]]['high'].values
low = stock_data[stock_symbols[1]]['low'].values
avg = stock_data[stock_symbols[1]]['avg'].values
volume = stock_data[stock_symbols[1]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_vcb = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[1]}'), ('close_week2', f'{stock_symbols[1]}'),
                                  ('close_week3', f'{stock_symbols[1]}'), ('close_week4', f'{stock_symbols[1]}')], dtype = np.dtype(np.float64))
df_vcb.head()

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)"
0,27.56,28.26,28.56,28.87
1,29.71,30.20,30.86,31.00
2,31.29,29.70,28.63,25.37
3,26.87,28.60,28.79,27.53
4,26.16,25.97,25.96,25.76


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_vcb[('close_total4week', f'{stock_symbols[1]}')] = mean_close[:len(df_vcb)]
df_vcb[('open_total4week', f'{stock_symbols[1]}')] = mean_open[:len(df_vcb)]
df_vcb[('high_total4week', f'{stock_symbols[1]}')] = mean_high[:len(df_vcb)]
df_vcb[('low_total4week', f'{stock_symbols[1]}')] = mean_low[:len(df_vcb)]
df_vcb[('avg_total4week', f'{stock_symbols[1]}')] = mean_avg[:len(df_vcb)]
df_vcb[('volume_total4week', f'{stock_symbols[1]}')] = mean_volume[:len(df_vcb)]

In [ ]:
df_vcb

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)","(close_total4week, VCB)","(open_total4week, VCB)","(high_total4week, VCB)","(low_total4week, VCB)","(avg_total4week, VCB)","(volume_total4week, VCB)"
0,27.56,28.26,28.56,28.87,28.31,28.30,28.57,28.02,28.31,1135513.93
1,29.71,30.20,30.86,31.00,30.44,30.42,30.83,30.14,30.48,1103760.71
2,31.29,29.70,28.63,25.37,28.75,28.90,29.25,28.38,28.74,780531.43
3,26.87,28.60,28.79,27.53,27.95,27.82,28.20,27.49,27.90,547587.50
4,26.16,25.97,25.96,25.76,25.96,25.98,26.14,25.81,25.96,301797.14
...,...,...,...,...,...,...,...,...,...,...
84,102.17,100.47,101.91,104.93,102.37,102.20,103.37,101.37,102.37,876050.00
85,96.00,90.73,89.27,87.59,90.90,91.05,91.67,90.31,90.91,1370264.39
86,89.43,88.97,87.99,85.67,88.01,88.05,88.65,87.15,87.94,1487303.57
87,85.93,85.21,88.54,86.67,86.59,86.36,87.12,85.54,86.31,1300789.29


##VNM

In [ ]:
close = stock_data[stock_symbols[2]]['close'].values
open = stock_data[stock_symbols[2]]['open'].values
high = stock_data[stock_symbols[2]]['high'].values
low = stock_data[stock_symbols[2]]['low'].values
avg = stock_data[stock_symbols[2]]['avg'].values
volume = stock_data[stock_symbols[2]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_vnm = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[2]}'), ('close_week2', f'{stock_symbols[2]}'),
                                  ('close_week3', f'{stock_symbols[2]}'), ('close_week4', f'{stock_symbols[2]}')], dtype = np.dtype(np.float64))
df_vnm.head()

,"(close_week1, VNM)","(close_week2, VNM)","(close_week3, VNM)","(close_week4, VNM)"
0,136.00,138.71,140.57,139.57
1,139.43,139.57,145.86,141.57
2,144.29,141.00,139.14,127.29
3,123.14,123.43,123.29,121.57
4,122.43,128.14,132.00,135.86


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_vnm[('close_total4week', f'{stock_symbols[2]}')] = mean_close[:len(df_vnm)]
df_vnm[('open_total4week', f'{stock_symbols[2]}')] = mean_open[:len(df_vnm)]
df_vnm[('high_total4week', f'{stock_symbols[2]}')] = mean_high[:len(df_vnm)]
df_vnm[('low_total4week', f'{stock_symbols[2]}')] = mean_low[:len(df_vnm)]
df_vnm[('avg_total4week', f'{stock_symbols[2]}')] = mean_avg[:len(df_vnm)]
df_vnm[('volume_total4week', f'{stock_symbols[2]}')] = mean_volume[:len(df_vnm)]

In [ ]:
df_vnm

,"(close_week1, VNM)","(close_week2, VNM)","(close_week3, VNM)","(close_week4, VNM)","(close_total4week, VNM)","(open_total4week, VNM)","(high_total4week, VNM)","(low_total4week, VNM)","(avg_total4week, VNM)","(volume_total4week, VNM)"
0,136.00,138.71,140.57,139.57,138.71,138.61,139.82,137.79,138.74,173054.64
1,139.43,139.57,145.86,141.57,141.61,141.71,143.18,140.50,141.78,368953.57
2,144.29,141.00,139.14,127.29,137.93,138.64,139.82,136.68,137.99,281483.21
3,123.14,123.43,123.29,121.57,122.86,122.96,124.14,121.96,122.93,130588.57
4,122.43,128.14,132.00,135.86,129.61,129.21,130.46,128.32,129.32,176452.86
...,...,...,...,...,...,...,...,...,...,...
84,66.39,70.09,71.06,72.83,70.09,70.02,70.61,69.46,69.98,5269664.29
85,75.39,74.31,73.26,74.86,74.45,74.16,74.96,73.49,74.28,3663646.43
86,79.24,79.20,76.40,74.14,77.25,77.13,78.01,76.39,77.18,2595932.14
87,74.43,69.20,70.36,69.84,70.96,71.17,71.78,70.23,70.97,2294228.57


##GAS

In [ ]:
close = stock_data[stock_symbols[3]]['close'].values
open = stock_data[stock_symbols[3]]['open'].values
high = stock_data[stock_symbols[3]]['high'].values
low = stock_data[stock_symbols[3]]['low'].values
avg = stock_data[stock_symbols[3]]['avg'].values
volume = stock_data[stock_symbols[3]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_gas = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[3]}'), ('close_week2', f'{stock_symbols[3]}'),
                                  ('close_week3', f'{stock_symbols[3]}'), ('close_week4', f'{stock_symbols[3]}')], dtype = np.dtype(np.float64))
df_gas.head()

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)"
0,67.36,75.07,76.50,81.36
1,83.71,83.00,84.93,83.00
2,87.36,92.14,95.43,93.79
3,90.36,95.36,103.14,105.29
4,109.29,110.57,110.71,112.43


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_gas[('close_total4week', f'{stock_symbols[3]}')] = mean_close[:len(df_gas)]
df_gas[('open_total4week', f'{stock_symbols[3]}')] = mean_open[:len(df_gas)]
df_gas[('high_total4week', f'{stock_symbols[3]}')] = mean_high[:len(df_gas)]
df_gas[('low_total4week', f'{stock_symbols[3]}')] = mean_low[:len(df_gas)]
df_gas[('avg_total4week', f'{stock_symbols[3]}')] = mean_avg[:len(df_gas)]
df_gas[('volume_total4week', f'{stock_symbols[3]}')] = mean_volume[:len(df_gas)]

In [ ]:
df_gas

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)","(close_total4week, GAS)","(open_total4week, GAS)","(high_total4week, GAS)","(low_total4week, GAS)","(avg_total4week, GAS)","(volume_total4week, GAS)"
0,67.36,75.07,76.50,81.36,75.07,74.93,76.04,74.07,75.06,544727.50
1,83.71,83.00,84.93,83.00,83.66,84.07,84.75,83.04,83.89,325925.36
2,87.36,92.14,95.43,93.79,92.18,92.07,93.43,90.70,91.96,521151.43
3,90.36,95.36,103.14,105.29,98.54,98.04,99.73,96.80,98.28,401566.43
4,109.29,110.57,110.71,112.43,110.75,110.43,111.75,109.21,110.43,277294.64
...,...,...,...,...,...,...,...,...,...,...
84,95.46,95.16,95.61,98.74,96.24,96.38,97.13,95.62,96.35,716707.14
85,99.17,100.59,100.33,101.17,100.31,100.44,101.10,99.70,100.36,538317.86
86,100.79,107.41,92.40,86.11,96.68,96.86,97.79,95.51,96.64,617839.29
87,84.97,79.30,78.51,79.37,80.54,80.82,81.49,79.71,80.47,546039.29


##HVN

In [ ]:
close = stock_data[stock_symbols[4]]['close'].values
open = stock_data[stock_symbols[4]]['open'].values
high = stock_data[stock_symbols[4]]['high'].values
low = stock_data[stock_symbols[4]]['low'].values
avg = stock_data[stock_symbols[4]]['avg'].values
volume = stock_data[stock_symbols[4]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_hvn = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[4]}'), ('close_week2', f'{stock_symbols[4]}'),
                                  ('close_week3', f'{stock_symbols[4]}'), ('close_week4', f'{stock_symbols[4]}')], dtype = np.dtype(np.float64))
df_hvn.head()

,"(close_week1, HVN)","(close_week2, HVN)","(close_week3, HVN)","(close_week4, HVN)"
0,43.69,40.24,38.47,41.14
1,38.44,33.83,31.19,30.46
2,28.30,25.79,26.33,26.19
3,28.20,26.83,26.77,26.97
4,26.73,26.66,26.47,26.27


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_hvn[('close_total4week', f'{stock_symbols[4]}')] = mean_close[:len(df_hvn)]
df_hvn[('open_total4week', f'{stock_symbols[4]}')] = mean_open[:len(df_hvn)]
df_hvn[('high_total4week', f'{stock_symbols[4]}')] = mean_high[:len(df_hvn)]
df_hvn[('low_total4week', f'{stock_symbols[4]}')] = mean_low[:len(df_hvn)]
df_hvn[('avg_total4week', f'{stock_symbols[4]}')] = mean_avg[:len(df_hvn)]
df_hvn[('volume_total4week', f'{stock_symbols[4]}')] = mean_volume[:len(df_hvn)]

In [ ]:
df_hvn

,"(close_week1, HVN)","(close_week2, HVN)","(close_week3, HVN)","(close_week4, HVN)","(close_total4week, HVN)","(open_total4week, HVN)","(high_total4week, HVN)","(low_total4week, HVN)","(avg_total4week, HVN)","(volume_total4week, HVN)"
0,43.69,40.24,38.47,41.14,40.89,41.42,41.99,40.08,40.95,184819.00
1,38.44,33.83,31.19,30.46,33.48,33.97,34.66,32.58,33.48,1792022.89
2,28.30,25.79,26.33,26.19,26.65,26.86,27.26,26.30,26.70,593148.07
3,28.20,26.83,26.77,26.97,27.19,27.17,27.57,26.88,27.20,433548.82
4,26.73,26.66,26.47,26.27,26.53,26.49,26.70,26.31,26.53,286891.11
...,...,...,...,...,...,...,...,...,...,...
57,13.46,13.51,14.29,13.20,13.61,13.65,13.85,13.49,13.63,2670992.86
58,12.99,12.97,13.24,13.04,13.06,13.04,13.24,12.82,13.05,2179442.86
59,12.75,13.24,12.67,11.81,12.62,12.68,12.83,12.40,12.64,2129121.43
60,11.64,11.34,10.64,10.79,11.10,11.13,11.29,10.94,11.10,1135360.71


##MWG

In [ ]:
close = stock_data[stock_symbols[5]]['close'].values
open = stock_data[stock_symbols[5]]['open'].values
high = stock_data[stock_symbols[5]]['high'].values
low = stock_data[stock_symbols[5]]['low'].values
avg = stock_data[stock_symbols[5]]['avg'].values
volume = stock_data[stock_symbols[5]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_mwg = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[5]}'), ('close_week2', f'{stock_symbols[5]}'),
                                  ('close_week3', f'{stock_symbols[5]}'), ('close_week4', f'{stock_symbols[5]}')], dtype = np.dtype(np.float64))
df_mwg.head()

,"(close_week1, MWG)","(close_week2, MWG)","(close_week3, MWG)","(close_week4, MWG)"
0,97.43,106.86,106.43,134.57
1,137.57,143.00,157.14,168.86
2,148.71,109.86,106.71,102.00
3,100.57,93.71,95.71,99.57
4,104.86,107.86,116.86,120.43


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_mwg[('close_total4week', f'{stock_symbols[5]}')] = mean_close[:len(df_mwg)]
df_mwg[('open_total4week', f'{stock_symbols[5]}')] = mean_open[:len(df_mwg)]
df_mwg[('high_total4week', f'{stock_symbols[5]}')] = mean_high[:len(df_mwg)]
df_mwg[('low_total4week', f'{stock_symbols[5]}')] = mean_low[:len(df_mwg)]
df_mwg[('avg_total4week', f'{stock_symbols[5]}')] = mean_avg[:len(df_mwg)]
df_mwg[('volume_total4week', f'{stock_symbols[5]}')] = mean_volume[:len(df_mwg)]

In [ ]:
df_mwg

,"(close_week1, MWG)","(close_week2, MWG)","(close_week3, MWG)","(close_week4, MWG)","(close_total4week, MWG)","(open_total4week, MWG)","(high_total4week, MWG)","(low_total4week, MWG)","(avg_total4week, MWG)","(volume_total4week, MWG)"
0,97.43,106.86,106.43,134.57,111.32,110.73,112.39,109.16,110.83,83082.50
1,137.57,143.00,157.14,168.86,151.64,152.32,154.07,148.86,151.47,34384.29
2,148.71,109.86,106.71,102.00,116.82,117.21,118.54,115.36,116.77,59553.93
3,100.57,93.71,95.71,99.57,97.39,97.54,98.54,96.23,97.40,74674.29
4,104.86,107.86,116.86,120.43,112.50,111.46,113.32,110.57,112.10,102131.43
...,...,...,...,...,...,...,...,...,...,...
79,38.68,41.52,42.11,43.49,41.45,41.38,41.86,40.89,41.35,3543671.43
80,44.98,48.89,53.64,52.73,50.06,49.78,50.66,49.14,49.88,6166753.57
81,53.06,50.59,54.56,56.11,53.58,53.59,54.48,52.60,53.56,8084530.97
82,52.34,48.89,47.11,42.67,47.75,48.24,48.91,47.08,47.98,7730639.29


##FPT

In [ ]:
close = stock_data[stock_symbols[6]]['close'].values
open = stock_data[stock_symbols[6]]['open'].values
high = stock_data[stock_symbols[6]]['high'].values
low = stock_data[stock_symbols[6]]['low'].values
avg = stock_data[stock_symbols[6]]['avg'].values
volume = stock_data[stock_symbols[6]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_fpt = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[6]}'), ('close_week2', f'{stock_symbols[6]}'),
                                  ('close_week3', f'{stock_symbols[6]}'), ('close_week4', f'{stock_symbols[6]}')], dtype = np.dtype(np.float64))
df_fpt.head()

,"(close_week1, FPT)","(close_week2, FPT)","(close_week3, FPT)","(close_week4, FPT)"
0,47.87,50.76,54.64,56.79
1,61.07,62.93,69.21,69.79
2,69.07,69.14,66.93,48.13
3,44.47,45.67,45.29,46.47
4,46.63,49.26,50.60,50.80


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_fpt[('close_total4week', f'{stock_symbols[6]}')] = mean_close[:len(df_fpt)]
df_fpt[('open_total4week', f'{stock_symbols[6]}')] = mean_open[:len(df_fpt)]
df_fpt[('high_total4week', f'{stock_symbols[6]}')] = mean_high[:len(df_fpt)]
df_fpt[('low_total4week', f'{stock_symbols[6]}')] = mean_low[:len(df_fpt)]
df_fpt[('avg_total4week', f'{stock_symbols[6]}')] = mean_avg[:len(df_fpt)]
df_fpt[('volume_total4week', f'{stock_symbols[6]}')] = mean_volume[:len(df_fpt)]

In [ ]:
df_fpt

,"(close_week1, FPT)","(close_week2, FPT)","(close_week3, FPT)","(close_week4, FPT)","(close_total4week, FPT)","(open_total4week, FPT)","(high_total4week, FPT)","(low_total4week, FPT)","(avg_total4week, FPT)","(volume_total4week, FPT)"
0,47.87,50.76,54.64,56.79,52.51,52.23,53.12,51.66,52.39,659624.29
1,61.07,62.93,69.21,69.79,65.75,65.57,66.77,64.46,65.59,1151072.50
2,69.07,69.14,66.93,48.13,63.32,63.61,64.52,62.36,63.33,1168561.79
3,44.47,45.67,45.29,46.47,45.48,45.38,46.02,44.85,45.45,766297.86
4,46.63,49.26,50.60,50.80,49.32,49.18,49.96,48.79,49.35,1107976.43
...,...,...,...,...,...,...,...,...,...,...
84,84.74,86.17,78.34,77.99,81.81,81.65,82.33,81.23,81.77,1129992.86
85,82.81,83.83,83.91,89.59,85.04,84.52,85.68,83.85,84.78,2278878.96
86,97.43,97.73,94.63,93.44,95.81,95.68,96.89,94.42,95.54,3418422.71
87,94.91,89.37,87.69,92.06,91.01,91.49,92.35,90.03,91.13,3525649.18


##HPG

In [ ]:
close = stock_data[stock_symbols[7]]['close'].values
open = stock_data[stock_symbols[7]]['open'].values
high = stock_data[stock_symbols[7]]['high'].values
low = stock_data[stock_symbols[7]]['low'].values
avg = stock_data[stock_symbols[7]]['avg'].values
volume = stock_data[stock_symbols[7]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_hpg = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[7]}'), ('close_week2', f'{stock_symbols[7]}'),
                                  ('close_week3', f'{stock_symbols[7]}'), ('close_week4', f'{stock_symbols[7]}')], dtype = np.dtype(np.float64))
df_hpg.head()

,"(close_week1, HPG)","(close_week2, HPG)","(close_week3, HPG)","(close_week4, HPG)"
0,42.29,48.33,47.66,48.44
1,49.13,48.71,50.84,52.36
2,54.64,54.21,50.77,45.81
3,48.60,51.14,49.79,52.29
4,53.64,55.14,57.29,56.14


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_hpg[('close_total4week', f'{stock_symbols[7]}')] = mean_close[:len(df_hpg)]
df_hpg[('open_total4week', f'{stock_symbols[7]}')] = mean_open[:len(df_hpg)]
df_hpg[('high_total4week', f'{stock_symbols[7]}')] = mean_high[:len(df_hpg)]
df_hpg[('low_total4week', f'{stock_symbols[7]}')] = mean_low[:len(df_hpg)]
df_hpg[('avg_total4week', f'{stock_symbols[7]}')] = mean_avg[:len(df_hpg)]
df_hpg[('volume_total4week', f'{stock_symbols[7]}')] = mean_volume[:len(df_hpg)]

In [ ]:
df_hpg

,"(close_week1, HPG)","(close_week2, HPG)","(close_week3, HPG)","(close_week4, HPG)","(close_total4week, HPG)","(open_total4week, HPG)","(high_total4week, HPG)","(low_total4week, HPG)","(avg_total4week, HPG)","(volume_total4week, HPG)"
0,42.29,48.33,47.66,48.44,46.68,46.45,47.30,45.88,46.58,586278.57
1,49.13,48.71,50.84,52.36,50.26,50.09,50.96,49.43,50.21,930927.50
2,54.64,54.21,50.77,45.81,51.36,51.19,52.08,50.15,51.13,775278.57
3,48.60,51.14,49.79,52.29,50.45,50.19,51.09,49.52,50.30,557077.50
4,53.64,55.14,57.29,56.14,55.55,55.25,56.02,54.75,55.42,921949.64
...,...,...,...,...,...,...,...,...,...,...
84,23.53,25.69,26.98,27.38,25.89,25.75,26.11,25.48,25.78,24379576.43
85,28.31,27.39,27.67,26.30,27.42,27.47,27.74,27.08,27.39,27604978.99
86,28.14,28.10,26.76,25.37,27.09,27.13,27.56,26.69,27.12,28598482.39
87,25.04,23.65,25.11,26.87,25.17,25.23,25.61,24.76,25.17,24687150.00


##MSN

In [ ]:
close = stock_data[stock_symbols[8]]['close'].values
open = stock_data[stock_symbols[8]]['open'].values
high = stock_data[stock_symbols[8]]['high'].values
low = stock_data[stock_symbols[8]]['low'].values
avg = stock_data[stock_symbols[8]]['avg'].values
volume = stock_data[stock_symbols[8]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_msn = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[8]}'), ('close_week2', f'{stock_symbols[8]}'),
                                  ('close_week3', f'{stock_symbols[8]}'), ('close_week4', f'{stock_symbols[8]}')], dtype = np.dtype(np.float64))
df_msn.head()

,"(close_week1, MSN)","(close_week2, MSN)","(close_week3, MSN)","(close_week4, MSN)"
0,83.36,91.29,93.21,93.07
1,98.00,97.57,104.07,99.07
2,94.64,94.29,94.29,90.50
3,91.00,96.00,98.00,95.07
4,91.71,92.36,91.79,90.29


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_msn[('close_total4week', f'{stock_symbols[8]}')] = mean_close[:len(df_msn)]
df_msn[('open_total4week', f'{stock_symbols[8]}')] = mean_open[:len(df_msn)]
df_msn[('high_total4week', f'{stock_symbols[8]}')] = mean_high[:len(df_msn)]
df_msn[('low_total4week', f'{stock_symbols[8]}')] = mean_low[:len(df_msn)]
df_msn[('avg_total4week', f'{stock_symbols[8]}')] = mean_avg[:len(df_msn)]
df_msn[('volume_total4week', f'{stock_symbols[8]}')] = mean_volume[:len(df_msn)]

In [ ]:
df_msn

,"(close_week1, MSN)","(close_week2, MSN)","(close_week3, MSN)","(close_week4, MSN)","(close_total4week, MSN)","(open_total4week, MSN)","(high_total4week, MSN)","(low_total4week, MSN)","(avg_total4week, MSN)","(volume_total4week, MSN)"
0,83.36,91.29,93.21,93.07,90.23,89.98,91.27,89.11,90.22,243162.50
1,98.00,97.57,104.07,99.07,99.68,99.79,101.32,98.57,99.78,598548.93
2,94.64,94.29,94.29,90.50,93.43,93.48,95.12,91.96,93.42,174805.36
3,91.00,96.00,98.00,95.07,95.02,95.14,96.16,93.96,95.02,156626.43
4,91.71,92.36,91.79,90.29,91.54,91.77,92.32,90.91,91.58,159496.43
...,...,...,...,...,...,...,...,...,...,...
84,76.61,76.66,76.44,80.66,77.59,77.65,78.31,76.91,77.59,2526742.86
85,84.66,87.10,82.14,79.14,83.26,83.37,84.26,82.31,83.23,2388935.77
86,81.24,79.51,75.19,73.87,77.45,77.64,78.72,76.51,77.53,3584410.71
87,70.99,63.59,62.01,63.57,65.04,65.43,66.29,63.90,64.95,2384225.00


##SAB

In [ ]:
close = stock_data[stock_symbols[9]]['close'].values
open = stock_data[stock_symbols[9]]['open'].values
high = stock_data[stock_symbols[9]]['high'].values
low = stock_data[stock_symbols[9]]['low'].values
avg = stock_data[stock_symbols[9]]['avg'].values
volume = stock_data[stock_symbols[9]]['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_sab = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[9]}'), ('close_week2', f'{stock_symbols[9]}'),
                                  ('close_week3', f'{stock_symbols[9]}'), ('close_week4', f'{stock_symbols[9]}')], dtype = np.dtype(np.float64))
df_sab.head()

,"(close_week1, SAB)","(close_week2, SAB)","(close_week3, SAB)","(close_week4, SAB)"
0,163.00,207.07,197.51,206.43
1,220.13,216.73,214.94,224.86
2,220.47,209.67,200.86,203.61
3,203.64,200.03,197.60,191.16
4,190.23,191.59,200.59,206.90


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_sab[('close_total4week', f'{stock_symbols[9]}')] = mean_close[:len(df_sab)]
df_sab[('open_total4week', f'{stock_symbols[9]}')] = mean_open[:len(df_sab)]
df_sab[('high_total4week', f'{stock_symbols[9]}')] = mean_high[:len(df_sab)]
df_sab[('low_total4week', f'{stock_symbols[9]}')] = mean_low[:len(df_sab)]
df_sab[('avg_total4week', f'{stock_symbols[9]}')] = mean_avg[:len(df_sab)]
df_sab[('volume_total4week', f'{stock_symbols[9]}')] = mean_volume[:len(df_sab)]

In [ ]:
df_sab

,"(close_week1, SAB)","(close_week2, SAB)","(close_week3, SAB)","(close_week4, SAB)","(close_total4week, SAB)","(open_total4week, SAB)","(high_total4week, SAB)","(low_total4week, SAB)","(avg_total4week, SAB)","(volume_total4week, SAB)"
0,163.00,207.07,197.51,206.43,193.50,193.66,195.70,189.88,193.21,115343.21
1,220.13,216.73,214.94,224.86,219.16,218.49,220.92,216.68,218.70,68125.36
2,220.47,209.67,200.86,203.61,208.65,209.16,210.87,206.50,208.56,31002.50
3,203.64,200.03,197.60,191.16,198.11,198.66,199.64,197.18,198.16,32071.43
4,190.23,191.59,200.59,206.90,197.32,196.91,198.14,195.04,196.66,39309.29
...,...,...,...,...,...,...,...,...,...,...
58,158.84,155.06,152.77,152.53,154.80,155.36,156.10,154.21,154.89,141939.29
59,156.87,159.81,158.49,153.20,157.09,157.62,158.58,156.20,157.27,248728.57
60,159.26,118.76,76.70,70.53,106.31,106.92,107.93,105.55,106.57,439900.00
61,69.27,67.67,61.64,63.16,65.44,65.74,66.64,64.49,65.50,603457.14


##All Company

In [ ]:
df_all = {}

In [ ]:
df_all[stock_symbols[0]] = df_vic
df_all[stock_symbols[1]] = df_vcb
df_all[stock_symbols[2]] = df_vnm
df_all[stock_symbols[3]] = df_gas
df_all[stock_symbols[4]] = df_hvn
df_all[stock_symbols[5]] = df_mwg
df_all[stock_symbols[6]] = df_fpt
df_all[stock_symbols[7]] = df_hpg
df_all[stock_symbols[8]] = df_msn
df_all[stock_symbols[9]] = df_sab

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]] = df_all[stock_symbols[i]].ewm(alpha=0.65).mean()

df_all[stock_symbols[2]]

,"(close_week1, VNM)","(close_week2, VNM)","(close_week3, VNM)","(close_week4, VNM)","(close_total4week, VNM)","(open_total4week, VNM)","(high_total4week, VNM)","(low_total4week, VNM)","(avg_total4week, VNM)","(volume_total4week, VNM)"
0,136.000000,138.710000,140.570000,139.570000,138.710000,138.610000,139.820000,137.790000,138.740000,1.730546e+05
1,138.540741,139.347037,144.488519,141.051481,140.858148,140.906296,142.308889,139.797407,140.991852,3.181650e+05
2,142.445161,140.469593,140.856248,131.705823,138.869593,139.367216,140.618642,137.680323,138.953243,2.932538e+05
3,129.705634,129.225120,129.264234,125.017167,128.304820,128.540050,129.744342,127.306440,128.379462,1.859106e+05
4,124.951502,128.516068,131.051871,132.102220,129.157666,128.977817,130.211976,127.968732,128.994040,1.797306e+05
...,...,...,...,...,...,...,...,...,...,...
84,68.259964,70.424983,70.480968,71.303389,70.116383,70.135998,70.680762,69.585443,70.069804,4.042388e+06
85,72.894487,72.950244,72.287339,73.615186,72.933234,72.751599,73.462267,72.123405,72.806431,3.796206e+06
86,77.019071,77.012585,74.960569,73.956315,75.739132,75.597560,76.418293,74.896692,75.649251,3.016028e+06
87,75.336175,71.934405,71.970199,71.280710,72.632696,72.719646,73.403403,71.863342,72.607738,2.546858e+06


In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['ema50', str(stock_symbols[i])] = df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')] / df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')].ewm(50).mean()
  df_all[stock_symbols[i]]['ema21', str(stock_symbols[i])] = df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')] / df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')].ewm(21).mean()
  df_all[stock_symbols[i]]['ema14', str(stock_symbols[i])] = df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')] / df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')].ewm(14).mean()
  df_all[stock_symbols[i]]['ema5', str(stock_symbols[i])] = df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')] / df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')].ewm(5).mean()

df_all[stock_symbols[3]].head()

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)","(close_total4week, GAS)","(open_total4week, GAS)","(high_total4week, GAS)","(low_total4week, GAS)","(avg_total4week, GAS)","(volume_total4week, GAS)","(ema50, GAS)","(ema21, GAS)","(ema14, GAS)","(ema5, GAS)"
0,67.360000,75.070000,76.500000,81.360000,75.070000,74.930000,76.040000,74.070000,75.060000,544727.500000,1.000000,1.000000,1.000000,1.000000
1,79.471111,80.944074,82.744444,82.574815,81.432963,81.700370,82.491852,80.714444,81.600741,382651.840741,1.040238,1.039674,1.039200,1.036825
2,84.828591,88.547419,91.359423,90.191239,88.731460,88.742564,89.920136,87.495806,88.635891,476709.286757,1.084273,1.082663,1.081313,1.074592
3,88.478782,93.043059,99.133457,100.154952,95.204142,94.877967,96.393691,93.635669,95.000064,427122.315376,1.116408,1.113458,1.110994,1.098866
4,102.077497,104.495722,106.697940,108.175860,105.362302,105.040162,106.427994,103.812434,105.082477,329220.125725,1.177826,1.172610,1.168261,1.147039


In [ ]:
def rsi(X, window=14):
  delta = X.diff(1)

  gains = delta.where(delta > 0, 0)
  losses = -delta.where(delta < 0, 0)

  avg_gains = gains.rolling(window=window, min_periods=1).mean()
  avg_losses = losses.rolling(window=window, min_periods=1).mean()

  rs = avg_gains / avg_losses
  rsi = 100 - (100 / (1 + rs))

  return rsi

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['rsi', str(stock_symbols[i])] = rsi(df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')])

df_all[stock_symbols[3]]

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)","(close_total4week, GAS)","(open_total4week, GAS)","(high_total4week, GAS)","(low_total4week, GAS)","(avg_total4week, GAS)","(volume_total4week, GAS)","(ema50, GAS)","(ema21, GAS)","(ema14, GAS)","(ema5, GAS)","(rsi, GAS)"
0,67.360000,75.070000,76.500000,81.360000,75.070000,74.930000,76.040000,74.070000,75.060000,544727.500000,1.000000,1.000000,1.000000,1.000000,NaN
1,79.471111,80.944074,82.744444,82.574815,81.432963,81.700370,82.491852,80.714444,81.600741,382651.840741,1.040238,1.039674,1.039200,1.036825,100.000000
2,84.828591,88.547419,91.359423,90.191239,88.731460,88.742564,89.920136,87.495806,88.635891,476709.286757,1.084273,1.082663,1.081313,1.074592,100.000000
3,88.478782,93.043059,99.133457,100.154952,95.204142,94.877967,96.393691,93.635669,95.000064,427122.315376,1.116408,1.113458,1.110994,1.098866,100.000000
4,102.077497,104.495722,106.697940,108.175860,105.362302,105.040162,106.427994,103.812434,105.082477,329220.125725,1.177826,1.172610,1.168261,1.147039,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,95.888422,95.683718,95.698619,97.513756,96.193884,96.414071,97.131079,95.607313,96.304797,597819.222239,1.062945,1.000051,0.970448,0.935980,34.490261
85,98.021448,98.872801,98.709017,99.890314,98.869359,99.030925,99.710878,98.267560,98.940679,559143.336784,1.090091,1.026541,0.997611,0.968143,40.814066
86,99.821007,104.421980,94.608156,90.933110,97.446276,97.619824,98.462307,96.475146,97.445238,597295.706374,1.072496,1.011215,0.984353,0.961546,17.425984
87,90.167852,88.092693,84.144355,83.417089,86.457197,86.699938,87.430308,85.577801,86.411333,563979.035731,0.952647,0.901464,0.880801,0.874521,11.670157


In [ ]:
def macd(X, short_window=12, long_window=29, signal=9):
  short_ema = X.ewm(span=short_window, adjust=False).mean()

  long_ema = X.ewm(span=long_window, adjust=False).mean()

  macd = short_ema - long_ema

  signal = macd.ewm(span=signal, adjust=False).mean()

  return signal

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['macd', str(stock_symbols[i])] = macd(df_all[stock_symbols[i]][('close_total4week',f'{stock_symbols[i]}')])

df_all[stock_symbols[5]].head()

,"(close_week1, MWG)","(close_week2, MWG)","(close_week3, MWG)","(close_week4, MWG)","(close_total4week, MWG)","(open_total4week, MWG)","(high_total4week, MWG)","(low_total4week, MWG)","(avg_total4week, MWG)","(volume_total4week, MWG)","(ema50, MWG)","(ema21, MWG)","(ema14, MWG)","(ema5, MWG)","(rsi, MWG)","(macd, MWG)"
0,97.430000,106.860000,106.430000,134.570000,111.320000,110.730000,112.390000,109.160000,110.830000,83082.500000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000
1,127.163333,133.630370,143.992963,159.970000,141.186667,141.537407,143.264074,138.567407,140.933704,47009.751852,1.116973,1.115213,1.113738,1.106384,100.000000,0.520752
2,141.796044,117.487504,118.673463,120.601579,124.638846,125.016248,126.473531,122.806859,124.523718,55528.718336,0.990754,0.989857,0.989127,0.985793,64.347724,1.054747
3,114.590868,101.796666,103.519813,106.722784,106.657260,106.884599,108.040120,105.268719,106.624704,68162.932088,0.882393,0.883214,0.883953,0.888303,46.379639,1.254584
4,108.232408,105.758642,112.236717,115.679517,110.475093,109.874312,111.490162,108.732745,110.202437,90359.033819,0.930629,0.932219,0.933600,0.941052,49.380693,1.297252


In [ ]:
def obv(X, num):
  obv = pd.Series(index=X.index)
  obv.iloc[0] = 0

  for i in range(1, len(X)):
    if (X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i].values > X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] + X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values
    elif (X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i].values < X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] - X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values
    else:
      obv.iloc[i] = obv.iloc[i-1]

  return obv

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['obv', str(stock_symbols[i])] = obv(df_all[stock_symbols[i]], i)


In [ ]:
df_all[stock_symbols[0]]

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)","(ema50, VIC)","(ema21, VIC)","(ema14, VIC)","(ema5, VIC)","(rsi, VIC)","(macd, VIC)","(obv, VIC)"
0,70.430000,72.930000,75.930000,77.430000,74.180000,74.120000,74.820000,73.200000,74.030000,2.594689e+05,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000e+00
1,75.822593,74.196667,76.826296,76.000370,75.713333,75.971852,76.516296,74.985185,75.726296,3.647390e+05,1.010127,1.009989,1.009873,1.009291,100.000000,0.026735,3.647390e+05
2,73.471087,69.166672,69.426435,67.463616,69.877453,70.021528,70.732733,69.019015,69.800119,5.717310e+05,0.954240,0.954749,0.955183,0.957443,20.807286,-0.032791,-2.069919e+05
3,68.587090,67.875459,66.643998,65.224160,67.081147,67.169740,67.800962,66.340460,67.035048,3.580046e+05,0.936291,0.937488,0.938503,0.943696,15.083668,-0.192514,-5.649965e+05
4,65.681219,68.747491,68.745437,69.788931,68.243506,68.319950,68.956908,67.562077,68.234064,4.044055e+05,0.962016,0.963496,0.964737,0.970856,23.796977,-0.386852,-1.605910e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,52.655544,52.625601,51.504168,52.236176,52.250608,52.414827,52.786729,51.897829,52.282568,2.828740e+06,0.642770,0.642366,0.668746,0.829014,4.799753,-10.151367,-2.210702e+07
85,52.255441,58.744960,64.351959,60.168662,58.880213,59.321189,59.945355,58.236740,59.034399,9.812150e+06,0.729144,0.733245,0.766217,0.944559,15.869197,-10.788884,-1.229487e+07
86,57.939404,56.479736,53.235686,50.692532,54.590074,55.244916,55.859874,53.981359,54.832540,1.317889e+07,0.681286,0.690041,0.724410,0.894257,18.055298,-11.276385,-2.547375e+07
87,49.437791,47.587908,45.665990,46.790886,47.368526,47.636721,48.137956,46.635476,47.349389,8.948605e+06,0.596962,0.610073,0.644578,0.806057,17.016013,-11.747391,-3.442236e+07


In [ ]:
def emv(X, num):
  emv = pd.Series(index=X.index)
  emv.iloc[0] = np.nan

  for i in range(1, len(X)):
    dm = 0.5 * ((X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i].values + X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i].values) - (X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values + X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values))
    br = X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values / (1000000 * (X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i].values - X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i].values))
    emv.iloc[i] = dm / br if br != 0 else 0

  return emv

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['emv', str(stock_symbols[i])] = emv(df_all[stock_symbols[i]], i)

In [ ]:
df_all[stock_symbols[7]].head()

,"(close_week1, HPG)","(close_week2, HPG)","(close_week3, HPG)","(close_week4, HPG)","(close_total4week, HPG)","(open_total4week, HPG)","(high_total4week, HPG)","(low_total4week, HPG)","(avg_total4week, HPG)","(volume_total4week, HPG)","(ema50, HPG)","(ema21, HPG)","(ema14, HPG)","(ema5, HPG)","(rsi, HPG)","(macd, HPG)","(obv, HPG)","(emv, HPG)"
0,42.290000,48.330000,47.660000,48.440000,46.680000,46.450000,47.300000,45.880000,46.580000,586278.570000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000e+00,NaN
1,47.356667,48.611481,50.015556,51.343704,49.331852,49.146296,50.011111,48.509630,49.268889,841574.073704,1.027339,1.026960,1.026642,1.025046,100.000000,0.046237,8.415741e+05,4.764300
2,52.302903,52.413531,50.527912,47.585671,50.709202,50.534211,51.416129,49.623633,50.532801,796551.660323,1.036286,1.035530,1.034897,1.031761,100.000000,0.143284,1.638126e+06,2.834302
3,49.859348,51.573125,50.040962,50.690070,50.538154,50.307065,51.200916,49.555245,50.379175,638522.023881,1.024144,1.023261,1.022529,1.019004,95.927668,0.264278,9.996037e+05,-0.365464
4,52.329747,53.903835,54.777715,54.251228,53.813054,53.536937,54.349859,52.949664,53.673011,823722.682945,1.070359,1.068605,1.067139,1.059921,97.711777,0.450881,1.823326e+06,5.561329


In [ ]:
def mfi(high, low, close, volume):
  combine_price = (high + low + close) / 3

  raw_money_flow = combine_price * volume

  flow_direction = (combine_price.diff() > 0).astype(int)

  positive_money_flow = flow_direction * raw_money_flow
  negative_money_flow = (1 - flow_direction) * raw_money_flow

  positive = positive_money_flow.rolling(window=14, min_periods=1).sum()
  negative = negative_money_flow.rolling(window=14, min_periods=1).sum()

  mf = positive / negative
  mfi = 100 - (100 / (1 + mf))

  return mfi

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['mfi', str(stock_symbols[i])] = mfi(df_all[stock_symbols[i]][('high_total4week', f'{stock_symbols[i]}')], df_all[stock_symbols[i]][('low_total4week', f'{stock_symbols[i]}')],
                                                               df_all[stock_symbols[i]][('close_total4week', f'{stock_symbols[i]}')], df_all[stock_symbols[i]][('volume_total4week', f'{stock_symbols[i]}')])

df_all[stock_symbols[8]].head()

,"(close_week1, MSN)","(close_week2, MSN)","(close_week3, MSN)","(close_week4, MSN)","(close_total4week, MSN)","(open_total4week, MSN)","(high_total4week, MSN)","(low_total4week, MSN)","(avg_total4week, MSN)","(volume_total4week, MSN)","(ema50, MSN)","(ema21, MSN)","(ema14, MSN)","(ema5, MSN)","(rsi, MSN)","(macd, MSN)","(obv, MSN)","(emv, MSN)","(mfi, MSN)"
0,83.360000,91.290000,93.210000,93.070000,90.230000,89.980000,91.270000,89.110000,90.220000,243162.500000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000,NaN,0.000000
1,94.204444,95.941852,101.254444,97.514444,97.230000,97.246667,98.714444,96.117407,97.301481,506411.707407,1.036958,1.036441,1.036007,1.033832,100.000000,0.122051,506411.707407,37.056800,69.208968
2,94.500238,94.820051,96.524771,92.750815,94.649355,94.688659,96.273396,93.294041,94.665501,281212.150594,1.006208,1.005797,1.005459,1.003854,73.063973,0.269834,225199.556813,-27.887418,50.370554
3,92.190422,95.598702,97.498278,94.281251,94.893945,94.986500,96.198566,93.733509,94.899436,198997.701633,1.006525,1.006135,1.005817,1.004363,73.734519,0.430157,424197.258446,2.258453,58.400521
4,91.876499,93.482431,93.768307,91.673240,92.702370,92.884736,93.664187,91.888538,92.730410,173186.308797,0.986711,0.986594,0.986513,0.986343,60.287132,0.551253,251010.949650,-22.450352,51.338250


In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]][[('volume_total4week', f'{stock_symbols[i]}')]] = df_all[stock_symbols[i]][[('volume_total4week', f'{stock_symbols[i]}')]] / df_all[stock_symbols[i]][[('volume_total4week', f'{stock_symbols[i]}')]].ewm(5).mean()

df_all[stock_symbols[0]]

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)","(ema50, VIC)","(ema21, VIC)","(ema14, VIC)","(ema5, VIC)","(rsi, VIC)","(macd, VIC)","(obv, VIC)","(emv, VIC)","(mfi, VIC)"
0,70.430000,72.930000,75.930000,77.430000,74.180000,74.120000,74.820000,73.200000,74.030000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000e+00,NaN,0.000000
1,75.822593,74.196667,76.826296,76.000370,75.713333,75.971852,76.516296,74.985185,75.726296,1.150999,1.010127,1.009989,1.009873,1.009291,100.000000,0.026735,3.647390e+05,7.307327,58.973335
2,73.471087,69.166672,69.426435,67.463616,69.877453,70.021528,70.732733,69.019015,69.800119,1.368741,0.954240,0.954749,0.955183,0.957443,20.807286,-0.032791,-2.069919e+05,-17.609446,31.828188
3,68.587090,67.875459,66.643998,65.224160,67.081147,67.169740,67.800962,66.340460,67.035048,0.898409,0.936291,0.937488,0.938503,0.943696,15.083668,-0.192514,-5.649965e+05,-11.443835,24.930677
4,65.681219,68.747491,68.745437,69.788931,68.243506,68.319950,68.956908,67.562077,68.234064,1.010669,0.962016,0.963496,0.964737,0.970856,23.796977,-0.386852,-1.605910e+05,4.100213,39.901517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,52.655544,52.625601,51.504168,52.236176,52.250608,52.414827,52.786729,51.897829,52.282568,1.138483,0.642770,0.642366,0.668746,0.829014,4.799753,-10.151367,-2.210702e+07,-0.144309,6.459576
85,52.255441,58.744960,64.351959,60.168662,58.880213,59.321189,59.945355,58.236740,59.034399,2.647706,0.729144,0.733245,0.766217,0.944559,15.869197,-10.788884,-1.229487e+07,1.175181,26.791567
86,57.939404,56.479736,53.235686,50.692532,54.590074,55.244916,55.859874,53.981359,54.832540,2.493764,0.681286,0.690041,0.724410,0.894257,18.055298,-11.276385,-2.547375e+07,-0.594452,23.585122
87,49.437791,47.587908,45.665990,46.790886,47.368526,47.636721,48.137956,46.635476,47.349389,1.517901,0.596962,0.610073,0.644578,0.806057,17.016013,-11.747391,-3.442236e+07,-1.264950,22.776413


In [ ]:
def closediff(x, y):
  res = []

  for i in range(len(x)):
    res.append(((y[i]-x[i]) / x[i])*100)

  return res

In [ ]:
for i in range(len(stock_symbols)):
  close_week1 = df_all[stock_symbols[i]][('close_week1', f'{stock_symbols[i]}')].values
  close_week2 = df_all[stock_symbols[i]][('close_week2', f'{stock_symbols[i]}')].values
  close_week3 = df_all[stock_symbols[i]][('close_week3', f'{stock_symbols[i]}')].values
  close_week4 = df_all[stock_symbols[i]][('close_week4', f'{stock_symbols[i]}')].values

  df_all[stock_symbols[i]][('closediff_week2_1', f'{stock_symbols[i]}')] = closediff(close_week1, close_week2)
  df_all[stock_symbols[i]][('closediff_week3_1', f'{stock_symbols[i]}')] = closediff(close_week1, close_week3)
  df_all[stock_symbols[i]][('closediff_week4_1', f'{stock_symbols[i]}')] = closediff(close_week1, close_week4)
  df_all[stock_symbols[i]][('closediff_week3_2', f'{stock_symbols[i]}')] = closediff(close_week2, close_week3)
  df_all[stock_symbols[i]][('closediff_week4_2', f'{stock_symbols[i]}')] = closediff(close_week2, close_week4)
  df_all[stock_symbols[i]][('closediff_week4_3', f'{stock_symbols[i]}')] = closediff(close_week3, close_week4)

In [ ]:
df_all[stock_symbols[0]]

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)",...,"(macd, VIC)","(obv, VIC)","(emv, VIC)","(mfi, VIC)","(closediff_week2_1, VIC)","(closediff_week3_1, VIC)","(closediff_week4_1, VIC)","(closediff_week3_2, VIC)","(closediff_week4_2, VIC)","(closediff_week4_3, VIC)"
0,70.430000,72.930000,75.930000,77.430000,74.180000,74.120000,74.820000,73.200000,74.030000,1.000000,...,0.000000,0.000000e+00,NaN,0.000000,3.549624,7.809172,9.938946,4.113534,6.170300,1.975504
1,75.822593,74.196667,76.826296,76.000370,75.713333,75.971852,76.516296,74.985185,75.726296,1.150999,...,0.026735,3.647390e+05,7.307327,58.973335,-2.144382,1.323753,0.234465,3.544134,2.430977,-1.075056
2,73.471087,69.166672,69.426435,67.463616,69.877453,70.021528,70.732733,69.019015,69.800119,1.368741,...,-0.032791,-2.069919e+05,-17.609446,31.828188,-5.858651,-5.505093,-8.176645,0.375560,-2.462249,-2.827192
3,68.587090,67.875459,66.643998,65.224160,67.081147,67.169740,67.800962,66.340460,67.035048,0.898409,...,-0.192514,-5.649965e+05,-11.443835,24.930677,-1.037558,-2.833028,-4.903153,-1.814295,-3.906123,-2.130482
4,65.681219,68.747491,68.745437,69.788931,68.243506,68.319950,68.956908,67.562077,68.234064,1.010669,...,-0.386852,-1.605910e+05,4.100213,39.901517,4.668414,4.665287,6.254013,-0.002988,1.514877,1.517911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,52.655544,52.625601,51.504168,52.236176,52.250608,52.414827,52.786729,51.897829,52.282568,1.138483,...,-10.151367,-2.210702e+07,-0.144309,6.459576,-0.056867,-2.186619,-0.796437,-2.130964,-0.739991,1.421260
85,52.255441,58.744960,64.351959,60.168662,58.880213,59.321189,59.945355,58.236740,59.034399,2.647706,...,-10.788884,-1.229487e+07,1.175181,26.791567,12.418841,23.148821,15.143344,9.544646,2.423529,-6.500652
86,57.939404,56.479736,53.235686,50.692532,54.590074,55.244916,55.859874,53.981359,54.832540,2.493764,...,-11.276385,-2.547375e+07,-0.594452,23.585122,-2.519301,-8.118341,-12.507675,-5.743742,-10.246515,-4.777160
87,49.437791,47.587908,45.665990,46.790886,47.368526,47.636721,48.137956,46.635476,47.349389,1.517901,...,-11.747391,-3.442236e+07,-1.264950,22.776413,-3.741842,-7.629389,-5.354012,-4.038668,-1.674841,2.463313


In [ ]:
def total_closediff(a, b, c, d, e, f):
  res = []

  for i in range(len(a)):
    res.append(((a[i]+b[i]+c[i]+d[i]+e[i]+f[i])/6))

  return res

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]][('total_closediff', f'{stock_symbols[i]}')] = total_closediff(df_all[stock_symbols[i]][('closediff_week2_1', f'{stock_symbols[i]}')], df_all[stock_symbols[i]][('closediff_week3_1', f'{stock_symbols[i]}')],
                                                                     df_all[stock_symbols[i]][('closediff_week4_1', f'{stock_symbols[i]}')], df_all[stock_symbols[i]][('closediff_week3_2', f'{stock_symbols[i]}')],
                                                                     df_all[stock_symbols[i]][('closediff_week4_2', f'{stock_symbols[i]}')], df_all[stock_symbols[i]][('closediff_week4_3', f'{stock_symbols[i]}')])

In [ ]:
df_all[stock_symbols[1]]

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)","(close_total4week, VCB)","(open_total4week, VCB)","(high_total4week, VCB)","(low_total4week, VCB)","(avg_total4week, VCB)","(volume_total4week, VCB)",...,"(obv, VCB)","(emv, VCB)","(mfi, VCB)","(closediff_week2_1, VCB)","(closediff_week3_1, VCB)","(closediff_week4_1, VCB)","(closediff_week3_2, VCB)","(closediff_week4_2, VCB)","(closediff_week4_3, VCB)","(total_closediff, VCB)"
0,27.560000,28.260000,28.560000,28.870000,28.310000,28.300000,28.570000,28.020000,28.310000,1.000000,...,0.000000e+00,NaN,0.000000,2.539913,3.628447,4.753266,1.061571,2.158528,1.085434,2.537860
1,29.152593,29.697037,30.263704,30.447778,29.887778,29.870370,30.244074,29.590370,29.917407,0.990477,...,1.111993e+06,0.953650,50.857740,1.867568,3.811363,4.442779,1.908159,2.527999,0.608234,2.527684
2,30.604143,29.699049,29.154228,26.999372,29.115093,29.211375,29.568981,28.768387,29.117810,0.861559,...,2.251013e+05,-0.675703,36.447639,-2.957421,-4.737643,-11.778702,-1.834475,-9.090114,-7.391229,-6.298264
3,28.139972,28.973784,28.913873,27.349535,28.346245,28.293203,28.665587,27.924776,28.314174,0.727394,...,-4.378827e+05,-0.976042,30.229075,2.963087,2.750183,-2.808948,-0.206776,-5.605926,-5.410337,-1.386453
4,26.846195,27.011015,26.983718,26.310882,26.786996,26.781682,27.015288,26.542914,26.775881,0.549903,...,-8.648557e+05,-1.677289,27.382894,0.613943,0.512262,-1.994000,-0.101061,-2.592030,-2.493489,-1.009062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,98.200197,97.604373,98.357439,101.516549,98.920337,98.689351,99.853124,97.844445,98.855633,0.796201,...,1.739009e+07,16.120648,53.229930,-0.606744,0.160123,3.377133,0.771549,4.008197,3.211867,1.820354
85,96.770069,93.136031,92.450604,92.464292,93.707118,93.723773,94.534093,92.947056,93.690972,1.121143,...,1.621423e+07,-6.894451,53.804693,-3.755333,-4.463638,-4.449492,-0.735942,-0.721245,0.014806,-2.351807
86,91.999024,90.428111,89.551211,88.048002,90.003991,90.035821,90.709433,89.178970,89.952840,1.248774,...,1.483593e+07,-4.215493,53.985013,-1.707533,-2.660694,-4.294635,-0.969720,-2.632045,-1.678603,-2.323872
87,88.054158,87.036339,88.893924,87.152301,87.784897,87.646537,88.376301,86.813639,87.584994,1.163730,...,1.350801e+07,-2.764519,53.566916,-1.155902,0.953692,-1.024208,2.134264,0.133234,-1.959215,-0.153022


In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]].replace(0, np.nan, inplace=True)
  df_all[stock_symbols[i]] = df_all[stock_symbols[i]].dropna()

In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]]['trend', str(stock_symbols[i])] = (df_all[stock_symbols[i]]['total_closediff', f'{stock_symbols[i]}'].iloc[:] > 0).astype(int)

<ipython-input-92-7a9993321a9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[stock_symbols[i]]['trend', str(stock_symbols[i])] = (df_all[stock_symbols[i]]['total_closediff', f'{stock_symbols[i]}'].iloc[:] > 0).astype(int)
<ipython-input-92-7a9993321a9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[stock_symbols[i]]['trend', str(stock_symbols[i])] = (df_all[stock_symbols[i]]['total_closediff', f'{stock_symbols[i]}'].iloc[:] > 0).astype(int)
<ipython-input-92-7a9993321a9e>:2: SettingWith

In [ ]:
df_all[stock_symbols[1]]

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)","(close_total4week, VCB)","(open_total4week, VCB)","(high_total4week, VCB)","(low_total4week, VCB)","(avg_total4week, VCB)","(volume_total4week, VCB)",...,"(emv, VCB)","(mfi, VCB)","(closediff_week2_1, VCB)","(closediff_week3_1, VCB)","(closediff_week4_1, VCB)","(closediff_week3_2, VCB)","(closediff_week4_2, VCB)","(closediff_week4_3, VCB)","(total_closediff, VCB)","(trend, VCB)"
1,29.152593,29.697037,30.263704,30.447778,29.887778,29.870370,30.244074,29.590370,29.917407,0.990477,...,0.953650,50.857740,1.867568,3.811363,4.442779,1.908159,2.527999,0.608234,2.527684,1
2,30.604143,29.699049,29.154228,26.999372,29.115093,29.211375,29.568981,28.768387,29.117810,0.861559,...,-0.675703,36.447639,-2.957421,-4.737643,-11.778702,-1.834475,-9.090114,-7.391229,-6.298264,0
3,28.139972,28.973784,28.913873,27.349535,28.346245,28.293203,28.665587,27.924776,28.314174,0.727394,...,-0.976042,30.229075,2.963087,2.750183,-2.808948,-0.206776,-5.605926,-5.410337,-1.386453,0
4,26.846195,27.011015,26.983718,26.310882,26.786996,26.781682,27.015288,26.542914,26.775881,0.549903,...,-1.677289,27.382894,0.613943,0.512262,-1.994000,-0.101061,-2.592030,-2.493489,-1.009062,0
5,26.249572,26.307061,27.345967,27.912252,26.951752,26.852218,27.122548,26.651720,26.895779,0.686168,...,0.105490,34.391866,0.219013,4.176814,6.334126,3.949152,6.101750,2.070818,3.808612,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,98.200197,97.604373,98.357439,101.516549,98.920337,98.689351,99.853124,97.844445,98.855633,0.796201,...,16.120648,53.229930,-0.606744,0.160123,3.377133,0.771549,4.008197,3.211867,1.820354,1
85,96.770069,93.136031,92.450604,92.464292,93.707118,93.723773,94.534093,92.947056,93.690972,1.121143,...,-6.894451,53.804693,-3.755333,-4.463638,-4.449492,-0.735942,-0.721245,0.014806,-2.351807,0
86,91.999024,90.428111,89.551211,88.048002,90.003991,90.035821,90.709433,89.178970,89.952840,1.248774,...,-4.215493,53.985013,-1.707533,-2.660694,-4.294635,-0.969720,-2.632045,-1.678603,-2.323872,0
87,88.054158,87.036339,88.893924,87.152301,87.784897,87.646537,88.376301,86.813639,87.584994,1.163730,...,-2.764519,53.566916,-1.155902,0.953692,-1.024208,2.134264,0.133234,-1.959215,-0.153022,0


In [ ]:
for i in range(len(stock_symbols)):
  df_all[stock_symbols[i]] = df_all[stock_symbols[i]][[('ema50', f'{stock_symbols[i]}'), ('ema21', f'{stock_symbols[i]}'), ('ema14', f'{stock_symbols[i]}'), ('ema5', f'{stock_symbols[i]}'), ('volume_total4week', f'{stock_symbols[i]}'),
                 ('rsi', f'{stock_symbols[i]}'), ('macd', f'{stock_symbols[i]}'), ('obv', f'{stock_symbols[i]}'), ('emv', f'{stock_symbols[i]}'),
                 ('mfi', f'{stock_symbols[i]}'), ('closediff_week2_1', f'{stock_symbols[i]}'), ('closediff_week3_1', f'{stock_symbols[i]}'), ('closediff_week4_1', f'{stock_symbols[i]}'),
                 ('closediff_week3_2', f'{stock_symbols[i]}'), ('closediff_week4_2', f'{stock_symbols[i]}'), ('closediff_week4_3', f'{stock_symbols[i]}'),
                 ('total_closediff', f'{stock_symbols[i]}'), ('trend', f'{stock_symbols[i]}')]]

In [ ]:
df_all[stock_symbols[1]]

,"(ema50, VCB)","(ema21, VCB)","(ema14, VCB)","(ema5, VCB)","(volume_total4week, VCB)","(rsi, VCB)","(macd, VCB)","(obv, VCB)","(emv, VCB)","(mfi, VCB)","(closediff_week2_1, VCB)","(closediff_week3_1, VCB)","(closediff_week4_1, VCB)","(closediff_week3_2, VCB)","(closediff_week4_2, VCB)","(closediff_week4_3, VCB)","(total_closediff, VCB)","(trend, VCB)"
1,1.026835,1.026463,1.026151,1.024585,0.990477,100.000000,0.027510,1.111993e+06,0.953650,50.857740,1.867568,3.811363,4.442779,1.908159,2.527999,0.608234,2.527684,1
2,1.000190,0.999952,0.999757,0.998849,0.861559,67.126278,0.057489,2.251013e+05,-0.675703,36.447639,-2.957421,-4.737643,-11.778702,-1.834475,-9.090114,-7.391229,-6.298264,0
3,0.980397,0.980490,0.980578,0.981167,0.727394,50.580980,0.073998,-4.378827e+05,-0.976042,30.229075,2.963087,2.750183,-2.808948,-0.206776,-5.605926,-5.410337,-1.386453,0
4,0.940858,0.941704,0.942438,0.946395,0.549903,33.723582,0.053829,-8.648557e+05,-1.677289,27.382894,0.613943,0.512262,-1.994000,-0.101061,-2.592030,-2.493489,-1.009062,0
5,0.955569,0.956873,0.957987,0.963756,0.686168,35.978110,0.014262,-3.826794e+05,0.105490,34.391866,0.219013,4.176814,6.334126,3.949152,6.101750,2.070818,3.808612,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1.369559,1.220621,1.169691,1.126987,0.796201,50.206288,2.736692,1.739009e+07,16.120648,53.229930,-0.606744,0.160123,3.377133,0.771549,4.008197,3.211867,1.820354,1
85,1.288198,1.147985,1.100102,1.055700,1.121143,54.858516,2.888951,1.621423e+07,-6.894451,53.804693,-3.755333,-4.463638,-4.449492,-0.735942,-0.721245,0.014806,-2.351807,0
86,1.230322,1.097409,1.052644,1.011624,1.248774,53.038629,2.989859,1.483593e+07,-4.215493,53.985013,-1.707533,-2.660694,-4.294635,-0.969720,-2.632045,-1.678603,-2.323872,0
87,1.194310,1.066882,1.024863,0.988877,1.163730,54.034785,3.008426,1.350801e+07,-2.764519,53.566916,-1.155902,0.953692,-1.024208,2.134264,0.133234,-1.959215,-0.153022,0


#LSTM Model

##VIC

In [ ]:
X = df_all[stock_symbols[0]].loc[:, df_all[stock_symbols[0]].columns != ('trend', 'VIC')]
y = df_all[stock_symbols[0]].loc[:, df_all[stock_symbols[0]].columns == ('trend', 'VIC')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(128,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_109"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_190 (LSTM)             (None, 10, 128)           74752     
                                                                 
 lstm_191 (LSTM)             (None, 128)               131584    
                                                                 
 dense_106 (Dense)           (None, 1)                 129       
                                                                 
Total params: 206465 (806.50 KB)
Trainable params: 206465 (806.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=100, batch_size=4)

Epoch 1/100
16/16 [==============================] - 8s 30ms/step - loss: 0.2640
Epoch 2/100
16/16 [==============================] - 0s 29ms/step - loss: 0.2545
Epoch 3/100
16/16 [==============================] - 0s 27ms/step - loss: 0.2474
Epoch 4/100
16/16 [==============================] - 0s 27ms/step - loss: 0.2473
Epoch 5/100
16/16 [==============================] - 0s 26ms/step - loss: 0.2484
Epoch 6/100
16/16 [==============================] - 0s 29ms/step - loss: 0.2463
Epoch 7/100
16/16 [==============================] - 0s 26ms/step - loss: 0.2453
Epoch 8/100
16/16 [==============================] - 0s 27ms/step - loss: 0.2397
Epoch 9/100
16/16 [==============================] - 0s 24ms/step - loss: 0.2358
Epoch 10/100
16/16 [==============================] - 0s 25ms/step - loss: 0.2355
Epoch 11/100
16/16 [==============================] - 0s 24ms/step - loss: 0.2406
Epoch 12/100
16/16 [==============================] - 1s 33ms/step - loss: 0.2284
Epoch 13/100
16/16 [=====

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=2)

9/9 [==============================] - 1s 5ms/step


In [ ]:
prediction = (y_pred > 0.5).astype(int)

In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy:  0.6111111111111112
Classification Report:
               precision    recall  f1-score   support

           0       0.53      1.00      0.70         8
           1       1.00      0.30      0.46        10

    accuracy                           0.61        18
   macro avg       0.77      0.65      0.58        18
weighted avg       0.79      0.61      0.57        18



##VCB

In [ ]:
X = df_all[stock_symbols[1]].loc[:, df_all[stock_symbols[1]].columns != ('trend', 'VCB')]
y = df_all[stock_symbols[1]].loc[:, df_all[stock_symbols[1]].columns == ('trend', 'VCB')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128,return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_68 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_69 (LSTM)              (None, 64)                49408     
                                                                 
 dense_34 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=20, batch_size=4)

Epoch 1/20
16/16 [==============================] - 10s 28ms/step - loss: 0.2113
Epoch 2/20
16/16 [==============================] - 0s 27ms/step - loss: 0.1873
Epoch 3/20
16/16 [==============================] - 0s 25ms/step - loss: 0.1797
Epoch 4/20
16/16 [==============================] - 0s 25ms/step - loss: 0.1707
Epoch 5/20
16/16 [==============================] - 0s 26ms/step - loss: 0.1683
Epoch 6/20
16/16 [==============================] - 0s 28ms/step - loss: 0.1604
Epoch 7/20
16/16 [==============================] - 0s 25ms/step - loss: 0.1698
Epoch 8/20
16/16 [==============================] - 0s 29ms/step - loss: 0.1581
Epoch 9/20
16/16 [==============================] - 0s 23ms/step - loss: 0.1723
Epoch 10/20
16/16 [==============================] - 0s 27ms/step - loss: 0.1473
Epoch 11/20
16/16 [==============================] - 0s 26ms/step - loss: 0.1404
Epoch 12/20
16/16 [==============================] - 0s 29ms/step - loss: 0.1405
Epoch 13/20
16/16 [=================

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=4)
prediction = (y_pred > 0.5).astype(int)

5/5 [==============================] - 1s 6ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6111111111111112
Classification Report:
               precision    recall  f1-score   support

           0       0.30      1.00      0.46         3
           1       1.00      0.53      0.70        15

    accuracy                           0.61        18
   macro avg       0.65      0.77      0.58        18
weighted avg       0.88      0.61      0.66        18



##VNM

In [ ]:
X = df_all[stock_symbols[2]].loc[:, df_all[stock_symbols[2]].columns != ('trend', 'VNM')]
y = df_all[stock_symbols[2]].loc[:, df_all[stock_symbols[2]].columns == ('trend', 'VNM')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_23 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=30, batch_size=8)

Epoch 1/30
8/8 [==============================] - 8s 21ms/step - loss: 0.2755
Epoch 2/30
8/8 [==============================] - 0s 23ms/step - loss: 0.2634
Epoch 3/30
8/8 [==============================] - 0s 24ms/step - loss: 0.2544
Epoch 4/30
8/8 [==============================] - 0s 21ms/step - loss: 0.2459
Epoch 5/30
8/8 [==============================] - 0s 25ms/step - loss: 0.2459
Epoch 6/30
8/8 [==============================] - 0s 22ms/step - loss: 0.2447
Epoch 7/30
8/8 [==============================] - 0s 22ms/step - loss: 0.2445
Epoch 8/30
8/8 [==============================] - 0s 22ms/step - loss: 0.2433
Epoch 9/30
8/8 [==============================] - 0s 22ms/step - loss: 0.2467
Epoch 10/30
8/8 [==============================] - 0s 20ms/step - loss: 0.2387
Epoch 11/30
8/8 [==============================] - 0s 22ms/step - loss: 0.2421
Epoch 12/30
8/8 [==============================] - 0s 23ms/step - loss: 0.2355
Epoch 13/30
8/8 [==============================] - 0s 27ms/st

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

3/3 [==============================] - 1s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6666666666666666
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.67      0.80        18
           1       0.00      0.00      0.00         0

    accuracy                           0.67        18
   macro avg       0.50      0.33      0.40        18
weighted avg       1.00      0.67      0.80        18



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##GAS

In [ ]:
X = df_all[stock_symbols[3]].loc[:, df_all[stock_symbols[3]].columns != ('trend', 'GAS')]
y = df_all[stock_symbols[3]].loc[:, df_all[stock_symbols[3]].columns == ('trend', 'GAS')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_58 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_59 (LSTM)              (None, 64)                49408     
                                                                 
 dense_30 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=4)

Epoch 1/10
16/16 [==============================] - 1s 41ms/step - loss: 0.1629
Epoch 2/10
16/16 [==============================] - 1s 40ms/step - loss: 0.1476
Epoch 3/10
16/16 [==============================] - 1s 39ms/step - loss: 0.1561
Epoch 4/10
16/16 [==============================] - 0s 20ms/step - loss: 0.1582
Epoch 5/10
16/16 [==============================] - 0s 22ms/step - loss: 0.1468
Epoch 6/10
16/16 [==============================] - 0s 28ms/step - loss: 0.1376
Epoch 7/10
16/16 [==============================] - 0s 27ms/step - loss: 0.1313
Epoch 8/10
16/16 [==============================] - 0s 20ms/step - loss: 0.1434
Epoch 9/10
16/16 [==============================] - 0s 21ms/step - loss: 0.1311
Epoch 10/10
16/16 [==============================] - 0s 21ms/step - loss: 0.1284


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=4)
prediction = (y_pred > 0.5).astype(int)

5/5 [==============================] - 0s 6ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6111111111111112
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.80      0.53         5
           1       0.88      0.54      0.67        13

    accuracy                           0.61        18
   macro avg       0.64      0.67      0.60        18
weighted avg       0.74      0.61      0.63        18



##HVN

In [ ]:
X = df_all[stock_symbols[4]].loc[:, df_all[stock_symbols[4]].columns != ('trend', 'HVN')]
y = df_all[stock_symbols[4]].loc[:, df_all[stock_symbols[4]].columns == ('trend', 'HVN')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (35, 10, 17)
y train dataset shape (35, 1)
X test dataset shape (12, 10, 17)
y test dataset shape (12, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_40 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_41 (LSTM)              (None, 64)                49408     
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=8)

Epoch 1/50
5/5 [==============================] - 7s 24ms/step - loss: 0.2332
Epoch 2/50
5/5 [==============================] - 0s 23ms/step - loss: 0.2304
Epoch 3/50
5/5 [==============================] - 0s 22ms/step - loss: 0.2212
Epoch 4/50
5/5 [==============================] - 0s 24ms/step - loss: 0.2113
Epoch 5/50
5/5 [==============================] - 0s 23ms/step - loss: 0.2101
Epoch 6/50
5/5 [==============================] - 0s 22ms/step - loss: 0.2110
Epoch 7/50
5/5 [==============================] - 0s 21ms/step - loss: 0.2113
Epoch 8/50
5/5 [==============================] - 0s 22ms/step - loss: 0.2114
Epoch 9/50
5/5 [==============================] - 0s 21ms/step - loss: 0.2091
Epoch 10/50
5/5 [==============================] - 0s 24ms/step - loss: 0.2090
Epoch 11/50
5/5 [==============================] - 0s 27ms/step - loss: 0.2102
Epoch 12/50
5/5 [==============================] - 0s 25ms/step - loss: 0.2143
Epoch 13/50
5/5 [==============================] - 0s 20ms/st

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

2/2 [==============================] - 1s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.8333333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.83      0.83      0.83         6

    accuracy                           0.83        12
   macro avg       0.83      0.83      0.83        12
weighted avg       0.83      0.83      0.83        12



##MWG

In [ ]:
X = df_all[stock_symbols[5]].loc[:, df_all[stock_symbols[5]].columns != ('trend', 'MWG')]
y = df_all[stock_symbols[5]].loc[:, df_all[stock_symbols[5]].columns == ('trend', 'MWG')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (57, 10, 17)
y train dataset shape (57, 1)
X test dataset shape (17, 10, 17)
y test dataset shape (17, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_80 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_81 (LSTM)              (None, 64)                49408     
                                                                 
 dense_41 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=4)

Epoch 1/50
15/15 [==============================] - 0s 20ms/step - loss: 0.2522
Epoch 2/50
15/15 [==============================] - 0s 18ms/step - loss: 0.2387
Epoch 3/50
15/15 [==============================] - 0s 18ms/step - loss: 0.2377
Epoch 4/50
15/15 [==============================] - 0s 19ms/step - loss: 0.2378
Epoch 5/50
15/15 [==============================] - 0s 26ms/step - loss: 0.2363
Epoch 6/50
15/15 [==============================] - 0s 33ms/step - loss: 0.2375
Epoch 7/50
15/15 [==============================] - 0s 27ms/step - loss: 0.2380
Epoch 8/50
15/15 [==============================] - 0s 27ms/step - loss: 0.2373
Epoch 9/50
15/15 [==============================] - 0s 30ms/step - loss: 0.2335
Epoch 10/50
15/15 [==============================] - 0s 32ms/step - loss: 0.2312
Epoch 11/50
15/15 [==============================] - 0s 31ms/step - loss: 0.2325
Epoch 12/50
15/15 [==============================] - 0s 30ms/step - loss: 0.2484
Epoch 13/50
15/15 [==================

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=4)
prediction = (y_pred > 0.5).astype(int)

5/5 [==============================] - 1s 7ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6470588235294118
Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         6
           1       1.00      0.45      0.62        11

    accuracy                           0.65        17
   macro avg       0.75      0.73      0.65        17
weighted avg       0.82      0.65      0.64        17



##FPT

In [ ]:
X = df_all[stock_symbols[6]].loc[:, df_all[stock_symbols[6]].columns != ('trend', 'FPT')]
y = df_all[stock_symbols[6]].loc[:, df_all[stock_symbols[6]].columns == ('trend', 'FPT')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_84 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_85 (LSTM)              (None, 64)                49408     
                                                                 
 dense_43 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=30, batch_size=4)

Epoch 1/30
16/16 [==============================] - 12s 29ms/step - loss: 0.2589
Epoch 2/30
16/16 [==============================] - 0s 24ms/step - loss: 0.2371
Epoch 3/30
16/16 [==============================] - 1s 34ms/step - loss: 0.2404
Epoch 4/30
16/16 [==============================] - 0s 29ms/step - loss: 0.2341
Epoch 5/30
16/16 [==============================] - 0s 30ms/step - loss: 0.2331
Epoch 6/30
16/16 [==============================] - 0s 25ms/step - loss: 0.2335
Epoch 7/30
16/16 [==============================] - 1s 40ms/step - loss: 0.2326
Epoch 8/30
16/16 [==============================] - 1s 46ms/step - loss: 0.2391
Epoch 9/30
16/16 [==============================] - 1s 36ms/step - loss: 0.2324
Epoch 10/30
16/16 [==============================] - 1s 32ms/step - loss: 0.2386
Epoch 11/30
16/16 [==============================] - 1s 31ms/step - loss: 0.2245
Epoch 12/30
16/16 [==============================] - 0s 31ms/step - loss: 0.2243
Epoch 13/30
16/16 [=================

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

3/3 [==============================] - 2s 10ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6111111111111112
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.64      0.67        11
           1       0.50      0.57      0.53         7

    accuracy                           0.61        18
   macro avg       0.60      0.60      0.60        18
weighted avg       0.62      0.61      0.61        18



##HPG

In [ ]:
X = df_all[stock_symbols[7]].loc[:, df_all[stock_symbols[7]].columns != ('trend', 'HPG')]
y = df_all[stock_symbols[7]].loc[:, df_all[stock_symbols[7]].columns == ('trend', 'HPG')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_99 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_100 (LSTM)             (None, 64)                49408     
                                                                 
 dense_61 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
8/8 [==============================] - 6s 23ms/step - loss: 0.2574
Epoch 2/15
8/8 [==============================] - 0s 25ms/step - loss: 0.2472
Epoch 3/15
8/8 [==============================] - 0s 24ms/step - loss: 0.2431
Epoch 4/15
8/8 [==============================] - 0s 23ms/step - loss: 0.2374
Epoch 5/15
8/8 [==============================] - 0s 37ms/step - loss: 0.2304
Epoch 6/15
8/8 [==============================] - 0s 32ms/step - loss: 0.2286
Epoch 7/15
8/8 [==============================] - 0s 30ms/step - loss: 0.2558
Epoch 8/15
8/8 [==============================] - 0s 29ms/step - loss: 0.2282
Epoch 9/15
8/8 [==============================] - 0s 30ms/step - loss: 0.2325
Epoch 10/15
8/8 [==============================] - 0s 31ms/step - loss: 0.2337
Epoch 11/15
8/8 [==============================] - 0s 25ms/step - loss: 0.2264
Epoch 12/15
8/8 [==============================] - 0s 36ms/step - loss: 0.2219
Epoch 13/15
8/8 [==============================] - 0s 37ms/st

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

3/3 [==============================] - 2s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.8333333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.70      1.00      0.82         7
           1       1.00      0.73      0.84        11

    accuracy                           0.83        18
   macro avg       0.85      0.86      0.83        18
weighted avg       0.88      0.83      0.83        18



##MSN

In [ ]:
X = df_all[stock_symbols[8]].loc[:, df_all[stock_symbols[8]].columns != ('trend', 'MSN')]
y = df_all[stock_symbols[8]].loc[:, df_all[stock_symbols[8]].columns == ('trend', 'MSN')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (61, 10, 17)
y train dataset shape (61, 1)
X test dataset shape (18, 10, 17)
y test dataset shape (18, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_101 (LSTM)             (None, 10, 128)           74752     
                                                                 
 lstm_102 (LSTM)             (None, 64)                49408     
                                                                 
 dense_62 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=100, batch_size=4)

Epoch 1/100
16/16 [==============================] - 6s 22ms/step - loss: 0.2657
Epoch 2/100
16/16 [==============================] - 0s 22ms/step - loss: 0.2480
Epoch 3/100
16/16 [==============================] - 0s 21ms/step - loss: 0.2397
Epoch 4/100
16/16 [==============================] - 0s 22ms/step - loss: 0.2312
Epoch 5/100
16/16 [==============================] - 0s 23ms/step - loss: 0.2388
Epoch 6/100
16/16 [==============================] - 0s 20ms/step - loss: 0.2364
Epoch 7/100
16/16 [==============================] - 0s 22ms/step - loss: 0.2118
Epoch 8/100
16/16 [==============================] - 0s 26ms/step - loss: 0.2207
Epoch 9/100
16/16 [==============================] - 0s 27ms/step - loss: 0.2199
Epoch 10/100
16/16 [==============================] - 0s 27ms/step - loss: 0.2172
Epoch 11/100
16/16 [==============================] - 1s 34ms/step - loss: 0.2149
Epoch 12/100
16/16 [==============================] - 1s 33ms/step - loss: 0.2018
Epoch 13/100
16/16 [=====

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

3/3 [==============================] - 1s 9ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6666666666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.75      0.80        16
           1       0.00      0.00      0.00         2

    accuracy                           0.67        18
   macro avg       0.43      0.38      0.40        18
weighted avg       0.76      0.67      0.71        18



##SAB

In [ ]:
X = df_all[stock_symbols[9]].loc[:, df_all[stock_symbols[9]].columns != ('trend', 'SAB')]
y = df_all[stock_symbols[9]].loc[:, df_all[stock_symbols[9]].columns == ('trend', 'SAB')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (40, 10, 17)
y train dataset shape (40, 1)
X test dataset shape (13, 10, 17)
y test dataset shape (13, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64,input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_114 (LSTM)             (None, 10, 128)           74752     
                                                                 
 lstm_115 (LSTM)             (None, 64)                49408     
                                                                 
 dense_58 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
5/5 [==============================] - 6s 21ms/step - loss: 0.2746
Epoch 2/15
5/5 [==============================] - 0s 20ms/step - loss: 0.2443
Epoch 3/15
5/5 [==============================] - 0s 19ms/step - loss: 0.2465
Epoch 4/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2469
Epoch 5/15
5/5 [==============================] - 0s 20ms/step - loss: 0.2464
Epoch 6/15
5/5 [==============================] - 0s 21ms/step - loss: 0.2447
Epoch 7/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2424
Epoch 8/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2422
Epoch 9/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2413
Epoch 10/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2385
Epoch 11/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2444
Epoch 12/15
5/5 [==============================] - 0s 19ms/step - loss: 0.2395
Epoch 13/15
5/5 [==============================] - 0s 18ms/st

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=2)
prediction = (y_pred > 0.5).astype(int)

7/7 [==============================] - 1s 6ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6153846153846154
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.62      0.76        13
           1       0.00      0.00      0.00         0

    accuracy                           0.62        13
   macro avg       0.50      0.31      0.38        13
weighted avg       1.00      0.62      0.76        13



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#THE END